# 🤖 Agent Loop - Ejercicio Desde Cero

## Objetivo
Crear un sistema de agente que resuelva problemas de planificación de viajes usando un loop.

## Herramientas que construiremos:
1. **add_destination**: Añadir destinos al itinerario
2. **calculate_cost**: Calcular costo estimado del viaje
3. **mark_visited**: Marcar un destino como visitado
4. **get_summary**: Obtener resumen del itinerario

---
## 📦 Paso 1: Importaciones

In [1]:
from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json

load_dotenv(override=True)

console = Console()
openai = OpenAI()

def show(text):
    """Helper para mostrar texto formateado"""
    try:
        console.print(text)
    except Exception:
        print(text)

---
## 🗺️ Paso 2: Estado Global del Itinerario

In [2]:
# Estado global del itinerario
destinations = []  # Lista de destinos
costs = []        # Costo estimado por destino
visited = []      # Si ya visitamos el destino

print("✅ Estado inicializado:")
print(f"Destinos: {destinations}")
print(f"Costos: {costs}")
print(f"Visitados: {visited}")

✅ Estado inicializado:
Destinos: []
Costos: []
Visitados: []


---
## 🛠️ Paso 3: Implementar las Herramientas (Tools)

In [3]:
def get_summary() -> str:
    """
    Genera un resumen del itinerario actual.
    Muestra destinos visitados en verde y tachados.
    """
    if not destinations:
        result = "[yellow]No hay destinos en el itinerario aún[/yellow]\n"
    else:
        result = "[bold cyan]📍 ITINERARIO DE VIAJE[/bold cyan]\n" + "=" * 50 + "\n"
        total_cost = 0
        
        for i, dest in enumerate(destinations):
            cost = costs[i]
            total_cost += cost
            
            if visited[i]:
                result += f"[green]✓ Destino #{i+1}: [strike]{dest}[/strike] - ${cost}[/green]\n"
            else:
                result += f"[white]○ Destino #{i+1}: {dest} - ${cost}[/white]\n"
        
        result += "=" * 50 + "\n"
        result += f"[bold yellow]💰 Costo total: ${total_cost}[/bold yellow]\n"
        visited_count = sum(visited)
        result += f"[bold magenta]📊 Progreso: {visited_count}/{len(destinations)} destinos visitados[/bold magenta]\n"
    
    show(result)
    return result

In [4]:
def add_destination(name: str, estimated_cost: float) -> str:
    """
    Añade un nuevo destino al itinerario.
    
    Args:
        name: Nombre del destino
        estimated_cost: Costo estimado en dólares
    
    Returns:
        Resumen actualizado del itinerario
    """
    destinations.append(name)
    costs.append(estimated_cost)
    visited.append(False)
    
    show(f"[green]✅ Añadido: {name} (${estimated_cost})[/green]")
    return get_summary()

In [5]:
def mark_visited(index: int, visit_notes: str) -> str:
    """
    Marca un destino como visitado.
    
    Args:
        index: Posición del destino (empieza en 1)
        visit_notes: Notas sobre la visita
    
    Returns:
        Resumen actualizado del itinerario
    """
    if 1 <= index <= len(destinations):
        visited[index - 1] = True
        show(f"[cyan]🎯 Visitado: {destinations[index-1]}[/cyan]")
        show(f"[italic]{visit_notes}[/italic]")
    else:
        show(f"[red]❌ Error: No existe destino #{index}[/red]")
        return "Error: Índice inválido"
    
    return get_summary()

In [6]:
def calculate_cost(items: list[dict]) -> str:
    """
    Calcula el costo de múltiples ítems.
    
    Args:
        items: Lista de diccionarios con 'name' y 'price'
    
    Returns:
        Descripción del cálculo y total
    """
    total = 0
    details = "[bold]💵 Desglose de costos:[/bold]\n"
    
    for item in items:
        name = item.get('name', 'Item')
        price = item.get('price', 0)
        total += price
        details += f"  • {name}: ${price}\n"
    
    details += f"[bold yellow]Total: ${total}[/bold yellow]"
    show(details)
    
    return f"Total calculado: ${total}"

---
## 🧪 Paso 4: Probar las Herramientas Manualmente

In [7]:
# Resetear estado
destinations, costs, visited = [], [], []

# Probar las funciones
print("\n=== Prueba 1: Añadir destinos ===")
add_destination("París", 1200)
add_destination("Roma", 900)
add_destination("Barcelona", 800)

print("\n=== Prueba 2: Marcar como visitado ===")
mark_visited(1, "Visitamos la Torre Eiffel y el Louvre. ¡Increíble!")

print("\n=== Prueba 3: Calcular costos ===")
calculate_cost([
    {"name": "Vuelo", "price": 500},
    {"name": "Hotel", "price": 300},
    {"name": "Comidas", "price": 200}
])


=== Prueba 1: Añadir destinos ===


✅ Añadido: París ($1200)

📍 ITINERARIO DE VIAJE
==================================================
○ Destino #1: París - $1200
==================================================
💰 Costo total: $1200
📊 Progreso: 0/1 destinos visitados

✅ Añadido: Roma ($900)

📍 ITINERARIO DE VIAJE
==================================================
○ Destino #1: París - $1200
○ Destino #2: Roma - $900
==================================================
💰 Costo total: $2100
📊 Progreso: 0/2 destinos visitados

...

Visitamos la Torre Eiffel y el Louvre. ¡Increíble!

📍 ITINERARIO DE VIAJE
==================================================
✓ Destino #1: París - $1200
○ Destino #2: Roma - $900
○ Destino #3: Barcelona - $800
==================================================
💰 Costo total: $2900
📊 Progreso: 1/3 destinos visitados


=== Prueba 3: Calcular costos ===


💵 Desglose de costos:
  • Vuelo: $500
  • Hotel: $300
  • Comidas: $200
Total: $1000

'Total calculado: $1000'

---
## 📋 Paso 5: Definir Tools en Formato JSON para OpenAI

In [8]:
# Definición de la herramienta add_destination
add_destination_json = {
    "name": "add_destination",
    "description": "Añade un nuevo destino al itinerario con su costo estimado",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Nombre del destino (ciudad, país, atracción)"
            },
            "estimated_cost": {
                "type": "number",
                "description": "Costo estimado total en dólares USD"
            }
        },
        "required": ["name", "estimated_cost"],
        "additionalProperties": False
    }
}

# Definición de la herramienta mark_visited
mark_visited_json = {
    "name": "mark_visited",
    "description": "Marca un destino como visitado con notas sobre la experiencia",
    "parameters": {
        "type": "object",
        "properties": {
            "index": {
                "type": "integer",
                "description": "Índice del destino (empezando en 1)"
            },
            "visit_notes": {
                "type": "string",
                "description": "Notas o resumen de la visita al destino"
            }
        },
        "required": ["index", "visit_notes"],
        "additionalProperties": False
    }
}

# Definición de la herramienta calculate_cost
calculate_cost_json = {
    "name": "calculate_cost",
    "description": "Calcula el costo total de múltiples ítems o gastos",
    "parameters": {
        "type": "object",
        "properties": {
            "items": {
                "type": "array",
                "description": "Lista de ítems con nombre y precio",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "price": {"type": "number"}
                    },
                    "required": ["name", "price"]
                }
            }
        },
        "required": ["items"],
        "additionalProperties": False
    }
}

# Definición de la herramienta get_summary
get_summary_json = {
    "name": "get_summary",
    "description": "Obtiene un resumen completo del itinerario actual",
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False
    }
}

# Array completo de tools en formato OpenAI
tools = [
    {"type": "function", "function": add_destination_json},
    {"type": "function", "function": mark_visited_json},
    {"type": "function", "function": calculate_cost_json},
    {"type": "function", "function": get_summary_json}
]

print("✅ Tools definidas:")
for tool in tools:
    print(f"  • {tool['function']['name']}")

✅ Tools definidas:
  • add_destination
  • mark_visited
  • calculate_cost
  • get_summary


---
## ⚙️ Paso 6: Handler de Tool Calls

In [9]:
def handle_tool_calls(tool_calls):
    """
    Procesa las llamadas a herramientas del LLM.
    
    Args:
        tool_calls: Lista de tool_calls del mensaje del LLM
    
    Returns:
        Lista de mensajes con los resultados de cada tool
    """
    results = []
    
    for tool_call in tool_calls:
        # Extraer información de la llamada
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        show(f"\n[bold blue]🔧 Ejecutando: {tool_name}[/bold blue]")
        show(f"[dim]Argumentos: {arguments}[/dim]")
        
        # Buscar la función en el scope global
        tool_function = globals().get(tool_name)
        
        if tool_function:
            # Ejecutar la función con los argumentos
            result = tool_function(**arguments)
        else:
            result = f"Error: Herramienta '{tool_name}' no encontrada"
            show(f"[red]{result}[/red]")
        
        # Crear mensaje de respuesta en formato OpenAI
        results.append({
            "role": "tool",
            "content": json.dumps(result) if not isinstance(result, str) else result,
            "tool_call_id": tool_call.id
        })
    
    return results

---
## 🔄 Paso 7: El Agent Loop (¡El Corazón del Sistema!)

In [10]:
def agent_loop(messages, max_iterations=10):
    """
    Loop principal del agente.
    
    El agente itera hasta que el LLM decide que terminó (finish_reason="stop")
    o alcanza el máximo de iteraciones.
    
    Args:
        messages: Lista de mensajes del chat
        max_iterations: Máximo número de iteraciones para evitar loops infinitos
    
    Returns:
        Respuesta final del LLM
    """
    iteration = 0
    done = False
    
    while not done and iteration < max_iterations:
        iteration += 1
        show(f"\n[bold magenta]{'='*60}[/bold magenta]")
        show(f"[bold magenta]🔁 ITERACIÓN {iteration}[/bold magenta]")
        show(f"[bold magenta]{'='*60}[/bold magenta]\n")
        
        # Llamar a la API de OpenAI
        response = openai.chat.completions.create(
            model="gpt-4",  # Usa "gpt-4" o "gpt-3.5-turbo" según tu acceso
            messages=messages,
            tools=tools,
            temperature=0.7
        )
        
        # Obtener información de la respuesta
        finish_reason = response.choices[0].finish_reason
        message = response.choices[0].message
        
        show(f"[yellow]📊 Finish reason: {finish_reason}[/yellow]")
        
        if finish_reason == "tool_calls":
            # El LLM quiere usar herramientas
            tool_calls = message.tool_calls
            show(f"[cyan]🛠️  El LLM solicitó {len(tool_calls)} herramienta(s)[/cyan]")
            
            # Ejecutar las herramientas
            results = handle_tool_calls(tool_calls)
            
            # Actualizar el historial de mensajes
            messages.append(message)  # Mensaje del asistente con tool_calls
            messages.extend(results)  # Resultados de las herramientas
            
        elif finish_reason == "stop":
            # El LLM terminó
            done = True
            show("[green]✅ El agente completó su tarea[/green]")
            
        else:
            # Otros finish_reasons (length, content_filter, etc.)
            show(f"[red]⚠️  Finish reason inesperado: {finish_reason}[/red]")
            done = True
    
    if iteration >= max_iterations:
        show(f"[red]⚠️  Se alcanzó el máximo de iteraciones ({max_iterations})[/red]")
    
    # Mostrar respuesta final
    final_content = message.content
    if final_content:
        show("\n" + "="*60)
        show("[bold green]💬 RESPUESTA FINAL DEL AGENTE:[/bold green]")
        show("="*60 + "\n")
        show(final_content)
    
    return final_content

---
## 🚀 Paso 8: Ejemplo 1 - Planificar un Viaje a Europa

In [11]:
# Resetear estado
destinations, costs, visited = [], [], []

system_message = """
Eres un asistente de planificación de viajes.
Tienes herramientas para:
- add_destination: Añadir destinos al itinerario
- mark_visited: Marcar destinos como visitados
- calculate_cost: Calcular costos de ítems
- get_summary: Obtener resumen del itinerario

Tu trabajo es:
1. Crear un plan de viaje basado en la solicitud del usuario
2. Añadir destinos con costos estimados realistas
3. Si el usuario menciona lugares ya visitados, marcarlos como tales
4. Proporcionar un resumen final con el costo total

Sé creativo pero realista con los costos. Usa formato Rich console markup en tus respuestas.
"""

user_message = """
Quiero planear un viaje de 2 semanas por Europa.
Me gustaría visitar París, Roma, Barcelona y Ámsterdam.
Ya he estado en París antes.
¿Puedes crear un itinerario con costos estimados?
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Ejecutar el agent loop
result = agent_loop(messages)

============================================================

🔁 ITERACIÓN 1

============================================================

📊 Finish reason: tool_calls

🛠️  El LLM solicitó 1 herramienta(s)

...

✅ El agente completó su tarea

============================================================

💬 RESPUESTA FINAL DEL AGENTE:

============================================================

Aquí tienes el itinerario de tu viaje de 2 semanas por Europa:

📍 ITINERARIO DE VIAJE
==================================================
✓ Destino #1: París - $2000 (Ya visitado en un viaje anterior)
○ Destino #2: Roma - $1800
○ Destino #3: Barcelona - $1600
○ Destino #4: Ámsterdam - $1700
==================================================
💰 Costo total: $7100
📊 Progreso: 1/4 destinos visitados

Los costos estimados incluyen alojamiento, comidas, transporte local y entradas a atracciones principales. Por 
favor, ten en cuenta que estos son solo estimaciones y los costos reales pueden variar.

---
## 🎯 Paso 9: Ejemplo 2 - Problema Matemático (Como en el Notebook Original)

In [12]:
# Resetear estado
destinations, costs, visited = [], [], []

system_message = """
Eres un agente que resuelve problemas usando herramientas de planificación.
Aunque las herramientas están diseñadas para viajes, úsalas creativamente para:
1. Descomponer el problema en pasos (usa add_destination para cada paso)
2. Marcar cada paso como completado cuando lo resuelvas (usa mark_visited)
3. Mostrar tu trabajo con get_summary
4. Dar la respuesta final

Usa Rich console markup. No uses bloques de código.
"""

user_message = """
Un tren sale de Madrid a las 10:00 am viajando a 120 km/h.
Otro tren sale de Barcelona a las 11:00 am viajando a 140 km/h hacia Madrid.
Si la distancia entre Madrid y Barcelona es de 620 km, ¿a qué hora se encuentran?
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Ejecutar el agent loop
result = agent_loop(messages)

============================================================

🔁 ITERACIÓN 1

============================================================

📊 Finish reason: tool_calls

🛠️  El LLM solicitó 1 herramienta(s)

...

✅ El agente completó su tarea

============================================================

💬 RESPUESTA FINAL DEL AGENTE:

============================================================

Los trenes que salen de Madrid y Barcelona se encuentran a las 12:55 pm.

---
## 📚 Conceptos Clave Aprendidos

### 1. **Agent Loop**
- Loop iterativo donde el LLM decide acciones
- Ejecuta herramientas hasta completar la tarea

### 2. **Tools (Herramientas)**
- Funciones que el LLM puede llamar
- Definidas en formato JSON para OpenAI
- Ejecutadas por el handler

### 3. **Estado Global**
- Variables compartidas entre iteraciones
- El LLM las modifica indirectamente via tools

### 4. **finish_reason**
- `"tool_calls"`: El LLM quiere usar herramientas
- `"stop"`: El LLM terminó
- `"length"`: Se alcanzó el límite de tokens

### 5. **Flujo de Mensajes**
```
[system] → [user] → [assistant + tool_calls] → [tool results] → [assistant + tool_calls] → ... → [assistant stop]
```

